In [1]:
!pip install xgboost
!pip install seaborn
!pip install pandas scikit-learn
!pip install mrmr_selection

In [2]:
import pandas as pd
import json
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
from collections import defaultdict
from pathlib import Path
import os
import itertools 
import numpy as np
import random
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import VarianceThreshold
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import mrmr
from mrmr import mrmr_classif

In [3]:
seed =1380
np.random.seed(seed)
random.seed(seed) 
os.environ['PYTHONHASHSEED']=str(seed)

# Early fusion. 132 subjects 

In [4]:
fmri132 = pd.read_csv('fmri_for_dti_AAL_CSA.csv')
fmri132

,"('Precentral_R', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_R')","('Frontal_Sup_R', 'Precentral_L')","('Frontal_Sup_R', 'Precentral_R')","('Frontal_Sup_R', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Precentral_L')","('Frontal_Sup_Orb_L', 'Precentral_R')","('Frontal_Sup_Orb_L', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Frontal_Sup_R')",...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,0.590307,0.215914,0.304414,0.116425,0.165778,0.255798,0.031173,-0.113507,0.038880,0.170122,...,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,1,A00000368
1,0.600053,0.450454,0.407950,0.554324,0.457576,0.747958,0.101570,-0.024283,0.218750,0.244049,...,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,1,A00000456
2,0.640427,0.425068,0.470462,0.313035,0.265838,0.615625,-0.134986,-0.242692,-0.040470,0.093006,...,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,1,A00000838
3,0.427308,0.239172,0.101873,0.266489,0.325015,0.580501,0.027387,-0.153969,0.092569,-0.007205,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1,A00000909
4,0.427308,0.239172,0.101873,0.266489,0.325015,0.580501,0.027387,-0.153969,0.092569,-0.007205,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1,A00000909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,0.787778,0.418824,0.420073,0.536880,0.531874,0.776564,0.222002,0.235727,-0.130599,0.042551,...,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,1,A00037649
128,0.837095,0.699350,0.567253,0.628090,0.484317,0.816644,0.515332,0.440825,0.597178,0.574797,...,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,0,A00037665
129,0.626127,0.367332,0.178448,0.252641,0.384334,0.654329,0.114585,0.223127,-0.013945,0.164300,...,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,1,A00037854
130,0.457744,0.486875,0.204815,0.321213,0.284081,0.572457,-0.062898,-0.092247,-0.064799,-0.135443,...,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,1,A00038441


In [18]:
target =fmri132.target
data =fmri132.drop(columns=['target', 'ID'])


# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Logistic Regression: Mean score using nested cross-validation is: 0.739 ± 0.107
Random Forest: Mean score using nested cross-validation is: 0.745 ± 0.099


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


SVC: Mean score using nested cross-validation is: 0.735 ± 0.101
XGBoost: Mean score using nested cross-validation is: 0.749 ± 0.135


# Drop duplicates from fmri (123 subj)

In [4]:
fmri_without_duplicates = fmri132.drop_duplicates()
fmri_without_duplicates.to_csv('fmri_without_duplicates.csv', index=False)

NameError: name 'fmri132' is not defined

In [5]:
fmri_123 = pd.read_csv('fmri_without_duplicates.csv')
fmri_123

,"('Precentral_R', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_R')","('Frontal_Sup_R', 'Precentral_L')","('Frontal_Sup_R', 'Precentral_R')","('Frontal_Sup_R', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Precentral_L')","('Frontal_Sup_Orb_L', 'Precentral_R')","('Frontal_Sup_Orb_L', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Frontal_Sup_R')",...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,0.590307,0.215914,0.304414,0.116425,0.165778,0.255798,0.031173,-0.113507,0.038880,0.170122,...,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,1,A00000368
1,0.600053,0.450454,0.407950,0.554324,0.457576,0.747958,0.101570,-0.024283,0.218750,0.244049,...,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,1,A00000456
2,0.640427,0.425068,0.470462,0.313035,0.265838,0.615625,-0.134986,-0.242692,-0.040470,0.093006,...,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,1,A00000838
3,0.427308,0.239172,0.101873,0.266489,0.325015,0.580501,0.027387,-0.153969,0.092569,-0.007205,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1,A00000909
4,0.664985,0.299166,0.073119,0.407197,0.240695,0.669216,0.168402,0.111856,0.097451,0.082961,...,-0.142375,0.048878,0.311628,0.185092,0.333022,0.338988,0.568694,0.423569,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0.787778,0.418824,0.420073,0.536880,0.531874,0.776564,0.222002,0.235727,-0.130599,0.042551,...,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,1,A00037649
119,0.837095,0.699350,0.567253,0.628090,0.484317,0.816644,0.515332,0.440825,0.597178,0.574797,...,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,0,A00037665
120,0.626127,0.367332,0.178448,0.252641,0.384334,0.654329,0.114585,0.223127,-0.013945,0.164300,...,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,1,A00037854
121,0.457744,0.486875,0.204815,0.321213,0.284081,0.572457,-0.062898,-0.092247,-0.064799,-0.135443,...,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,1,A00038441


# Early fusion. Only 75 regions of the brain

In [20]:
reg75= pd.read_csv('fmri_cobre_75regions_new.csv')
reg75

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,17 1ary Visual Anterior (BA17a),20 Posterior Fusiform (BA37m),27 2ary/3ary Visual Anterior (BA18/19a),29 Precuneus (BA7m),41 Angular (BA39),10 2ary/3ary Visual Posterior (BA18/19p),14 1ary Visual Posterior (BA17p),19 Posterior Middle Temporal (BA37-pMT),target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,103.948415,117.754574,97.951265,90.466617,74.563782,85.600885,116.860705,90.293591,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,135.452954,113.762617,99.168800,84.776504,84.182907,121.552500,116.242818,98.573632,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,171.634672,128.278143,165.282009,98.787972,107.424595,139.489700,146.718045,122.982998,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,148.392664,146.273071,145.256096,113.367215,130.451834,136.680432,139.514955,114.017828,1,A00000909
4,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,148.392664,146.273071,145.256096,113.367215,130.451834,136.680432,139.514955,114.017828,1,A00000909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,121.907310,140.090268,150.628304,120.923228,107.951374,133.114588,144.640263,93.140918,1,A00037649
128,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,130.509874,127.331203,119.756711,110.778300,87.421152,143.013958,117.791072,103.933863,0,A00037665
129,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,151.471291,124.232667,124.079747,75.842177,101.550097,124.739328,149.489989,104.270125,1,A00037854
130,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,114.575210,99.474787,135.220648,94.860538,94.225460,98.043687,121.035822,97.334191,1,A00038441


In [25]:
target =reg75.target
data =reg75.drop(columns=['target', 'ID'])


# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Logistic Regression: Mean score using nested cross-validation is: 0.664 ± 0.126
Random Forest: Mean score using nested cross-validation is: 0.615 ± 0.118
SVC: Mean score using nested cross-validation is: 0.664 ± 0.127
XGBoost: Mean score using nested cross-validation is: 0.725 ± 0.069


# Drop duplicates from 75 reg (123 subj)

In [33]:
reg75= pd.read_csv('fmri_cobre_75regions_new.csv')
reg75.drop_duplicates()
reg75

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,17 1ary Visual Anterior (BA17a),20 Posterior Fusiform (BA37m),27 2ary/3ary Visual Anterior (BA18/19a),29 Precuneus (BA7m),41 Angular (BA39),10 2ary/3ary Visual Posterior (BA18/19p),14 1ary Visual Posterior (BA17p),19 Posterior Middle Temporal (BA37-pMT),target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,103.948415,117.754574,97.951265,90.466617,74.563782,85.600885,116.860705,90.293591,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,135.452954,113.762617,99.168800,84.776504,84.182907,121.552500,116.242818,98.573632,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,171.634672,128.278143,165.282009,98.787972,107.424595,139.489700,146.718045,122.982998,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,148.392664,146.273071,145.256096,113.367215,130.451834,136.680432,139.514955,114.017828,1,A00000909
4,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,148.392664,146.273071,145.256096,113.367215,130.451834,136.680432,139.514955,114.017828,1,A00000909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,121.907310,140.090268,150.628304,120.923228,107.951374,133.114588,144.640263,93.140918,1,A00037649
128,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,130.509874,127.331203,119.756711,110.778300,87.421152,143.013958,117.791072,103.933863,0,A00037665
129,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,151.471291,124.232667,124.079747,75.842177,101.550097,124.739328,149.489989,104.270125,1,A00037854
130,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,114.575210,99.474787,135.220648,94.860538,94.225460,98.043687,121.035822,97.334191,1,A00038441


In [41]:
# Read the CSV files
file1 = pd.read_csv('fmri_cobre_75regions_new.csv')
file2 = pd.read_csv('fmri_without_duplicates.csv')

# Extract the ID column from each file
id_col_file1 = file1['ID']
id_col_file2 = file2['ID']

# Find the common IDs between the two files
common_ids = id_col_file1[id_col_file1.isin(id_col_file2)]

# Filter the rows with common IDs in both files
file1_filtered = file1[file1['ID'].isin(common_ids)]
file2_filtered = file2[file2['ID'].isin(common_ids)]

# Save the filtered files with different names
file1_filtered.to_csv('fmri_cobre_75regions_new.csv', index=False)
file2_filtered.to_csv('fmri_without_duplicates.csv', index=False)

# Print the IDs that are different between the two files (optional)
ids_different_file1 = id_col_file1[~id_col_file1.isin(id_col_file2)]
ids_different_file2 = id_col_file2[~id_col_file2.isin(id_col_file1)]
print("IDs in file1.csv that are different from file2.csv:")
print(ids_different_file1)

print("\nIDs in file2.csv that are different from file1.csv:")
print(ids_different_file2)

IDs in file1.csv that are different from file2.csv:
Series([], Name: ID, dtype: object)

IDs in file2.csv that are different from file1.csv:
Series([], Name: ID, dtype: object)


In [43]:
file1_filtered.drop_duplicates()

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,17 1ary Visual Anterior (BA17a),20 Posterior Fusiform (BA37m),27 2ary/3ary Visual Anterior (BA18/19a),29 Precuneus (BA7m),41 Angular (BA39),10 2ary/3ary Visual Posterior (BA18/19p),14 1ary Visual Posterior (BA17p),19 Posterior Middle Temporal (BA37-pMT),target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,103.948415,117.754574,97.951265,90.466617,74.563782,85.600885,116.860705,90.293591,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,135.452954,113.762617,99.168800,84.776504,84.182907,121.552500,116.242818,98.573632,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,171.634672,128.278143,165.282009,98.787972,107.424595,139.489700,146.718045,122.982998,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,148.392664,146.273071,145.256096,113.367215,130.451834,136.680432,139.514955,114.017828,1,A00000909
5,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,107.198542,79.971999,108.201993,103.729257,106.887096,118.010823,124.285205,94.770384,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,121.907310,140.090268,150.628304,120.923228,107.951374,133.114588,144.640263,93.140918,1,A00037649
128,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,130.509874,127.331203,119.756711,110.778300,87.421152,143.013958,117.791072,103.933863,0,A00037665
129,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,151.471291,124.232667,124.079747,75.842177,101.550097,124.739328,149.489989,104.270125,1,A00037854
130,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,114.575210,99.474787,135.220648,94.860538,94.225460,98.043687,121.035822,97.334191,1,A00038441


In [45]:
file1.drop_duplicates()
reg75_without_duplicates = file1.drop_duplicates()
reg75_without_duplicates.to_csv('75reg_without_duplicates.csv', index=False)

In [46]:
reg75_without_duplicates

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,17 1ary Visual Anterior (BA17a),20 Posterior Fusiform (BA37m),27 2ary/3ary Visual Anterior (BA18/19a),29 Precuneus (BA7m),41 Angular (BA39),10 2ary/3ary Visual Posterior (BA18/19p),14 1ary Visual Posterior (BA17p),19 Posterior Middle Temporal (BA37-pMT),target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,103.948415,117.754574,97.951265,90.466617,74.563782,85.600885,116.860705,90.293591,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,135.452954,113.762617,99.168800,84.776504,84.182907,121.552500,116.242818,98.573632,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,171.634672,128.278143,165.282009,98.787972,107.424595,139.489700,146.718045,122.982998,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,148.392664,146.273071,145.256096,113.367215,130.451834,136.680432,139.514955,114.017828,1,A00000909
5,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,107.198542,79.971999,108.201993,103.729257,106.887096,118.010823,124.285205,94.770384,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,121.907310,140.090268,150.628304,120.923228,107.951374,133.114588,144.640263,93.140918,1,A00037649
128,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,130.509874,127.331203,119.756711,110.778300,87.421152,143.013958,117.791072,103.933863,0,A00037665
129,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,151.471291,124.232667,124.079747,75.842177,101.550097,124.739328,149.489989,104.270125,1,A00037854
130,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,114.575210,99.474787,135.220648,94.860538,94.225460,98.043687,121.035822,97.334191,1,A00038441


In [47]:
target =reg75_without_duplicates.target
data =reg75_without_duplicates.drop(columns=['target', 'ID'])


# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Logistic Regression: Mean score using nested cross-validation is: 0.619 ± 0.083
Random Forest: Mean score using nested cross-validation is: 0.574 ± 0.116
SVC: Mean score using nested cross-validation is: 0.618 ± 0.111
XGBoost: Mean score using nested cross-validation is: 0.645 ± 0.102


# Check id differences b/w 75 regions data and fmri

In [29]:
# Read the CSV files
file1 = pd.read_csv('fmri_cobre_75regions_new.csv')
file2 = pd.read_csv('fmri_for_dti_AAL_CSA.csv')

# Extract the ID column from each file
id_col_file1 = file1['ID']
id_col_file2 = file2['ID']

# Find the common IDs between the two files
common_ids = id_col_file1[id_col_file1.isin(id_col_file2)]

# Filter the rows with common IDs in both files
file1_filtered = file1[file1['ID'].isin(common_ids)]
file2_filtered = file2[file2['ID'].isin(common_ids)]

# Save the filtered files with different names
file1_filtered.to_csv('fmri_cobre_75regions_new.csv', index=False)
file2_filtered.to_csv('fmri_for_dti_AAL_CSA.csv', index=False)

# Print the IDs that are different between the two files (optional)
ids_different_file1 = id_col_file1[~id_col_file1.isin(id_col_file2)]
ids_different_file2 = id_col_file2[~id_col_file2.isin(id_col_file1)]
print("IDs in file1.csv that are different from file2.csv:")
print(ids_different_file1)

print("\nIDs in file2.csv that are different from file1.csv:")
print(ids_different_file2)

IDs in file1.csv that are different from file2.csv:
Series([], Name: ID, dtype: object)

IDs in file2.csv that are different from file1.csv:
Series([], Name: ID, dtype: object)


In [30]:
# Read the CSV files
file1 = pd.read_csv('fmri_for_dti_AAL_CSA.csv')
file2 = pd.read_csv('fmri_cobre_75regions_new.csv')

# Extract the ID column from each file
id_col_file1 = file1['ID']
id_col_file2 = file2['ID']

# Find the common IDs between the two files
common_ids = id_col_file1[id_col_file1.isin(id_col_file2)]

# Filter the rows with common IDs in both files
file1_filtered = file1[file1['ID'].isin(common_ids)]
file2_filtered = file2[file2['ID'].isin(common_ids)]

# Save the filtered files with different names
file1_filtered.to_csv('fmri_for_dti_AAL_CSA.csv', index=False)
file2_filtered.to_csv('fmri_cobre_75regions_new.csv', index=False)

# Print the IDs that are different between the two files (optional)
ids_different_file1 = id_col_file1[~id_col_file1.isin(id_col_file2)]
ids_different_file2 = id_col_file2[~id_col_file2.isin(id_col_file1)]
print("IDs in file1.csv that are different from file2.csv:")
print(ids_different_file1)

print("\nIDs in file2.csv that are different from file1.csv:")
print(ids_different_file2)

IDs in file1.csv that are different from file2.csv:
Series([], Name: ID, dtype: object)

IDs in file2.csv that are different from file1.csv:
Series([], Name: ID, dtype: object)


# EF for fmri+75 regions

In [26]:
# Path to the CSV files

file1_path = 'fmri_cobre_75regions_new.csv'
file2_path = 'fmri_for_dti_AAL_CSA.csv'

# Read the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Concatenate the dataframes based on the ID column
concatenated_df = pd.merge(df1, df2, on='ID')

# Save the concatenated dataframe to a new CSV file
output_path = 'fmri_cobre_75regions.csv'
concatenated_df.to_csv(output_path, index=False)

print("CSV files concatenated based on the ID column and saved as 'concatenated.csv'.")

CSV files concatenated based on the ID column and saved as 'concatenated.csv'.


In [32]:
fmri_and_75reg = pd.read_csv('fmri_cobre_75regions.csv')
fmri_and_75reg = file3.drop_duplicates()
fmri_and_75reg

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,17 1ary Visual Anterior (BA17a),20 Posterior Fusiform (BA37m),27 2ary/3ary Visual Anterior (BA18/19a),29 Precuneus (BA7m),41 Angular (BA39),10 2ary/3ary Visual Posterior (BA18/19p),14 1ary Visual Posterior (BA17p),19 Posterior Middle Temporal (BA37-pMT),target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,103.948415,117.754574,97.951265,90.466617,74.563782,85.600885,116.860705,90.293591,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,135.452954,113.762617,99.168800,84.776504,84.182907,121.552500,116.242818,98.573632,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,171.634672,128.278143,165.282009,98.787972,107.424595,139.489700,146.718045,122.982998,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,148.392664,146.273071,145.256096,113.367215,130.451834,136.680432,139.514955,114.017828,1,A00000909
5,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,107.198542,79.971999,108.201993,103.729257,106.887096,118.010823,124.285205,94.770384,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,121.907310,140.090268,150.628304,120.923228,107.951374,133.114588,144.640263,93.140918,1,A00037649
128,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,130.509874,127.331203,119.756711,110.778300,87.421152,143.013958,117.791072,103.933863,0,A00037665
129,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,151.471291,124.232667,124.079747,75.842177,101.550097,124.739328,149.489989,104.270125,1,A00037854
130,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,114.575210,99.474787,135.220648,94.860538,94.225460,98.043687,121.035822,97.334191,1,A00038441


In [51]:
reg75_without_duplicates = reg75.drop_duplicates()

# Save the new DataFrame to a CSV file
reg75_without_duplicates.to_csv('75regions_without_duplicates.csv', index=False)

In [52]:
reg75_without_duplicates

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,17 1ary Visual Anterior (BA17a),20 Posterior Fusiform (BA37m),27 2ary/3ary Visual Anterior (BA18/19a),29 Precuneus (BA7m),41 Angular (BA39),10 2ary/3ary Visual Posterior (BA18/19p),14 1ary Visual Posterior (BA17p),19 Posterior Middle Temporal (BA37-pMT),target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,103.948415,117.754574,97.951265,90.466617,74.563782,85.600885,116.860705,90.293591,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,135.452954,113.762617,99.168800,84.776504,84.182907,121.552500,116.242818,98.573632,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,171.634672,128.278143,165.282009,98.787972,107.424595,139.489700,146.718045,122.982998,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,148.392664,146.273071,145.256096,113.367215,130.451834,136.680432,139.514955,114.017828,1,A00000909
4,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,107.198542,79.971999,108.201993,103.729257,106.887096,118.010823,124.285205,94.770384,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,121.907310,140.090268,150.628304,120.923228,107.951374,133.114588,144.640263,93.140918,1,A00037649
119,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,130.509874,127.331203,119.756711,110.778300,87.421152,143.013958,117.791072,103.933863,0,A00037665
120,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,151.471291,124.232667,124.079747,75.842177,101.550097,124.739328,149.489989,104.270125,1,A00037854
121,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,114.575210,99.474787,135.220648,94.860538,94.225460,98.043687,121.035822,97.334191,1,A00038441


In [26]:
target =file3.target
data =file3.drop(columns=['target', 'ID'])

# Define the pipeline for feature selection and model training
pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=seed))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=seed))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=seed))
])

# Define the parameter grids for each model
param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Random Forest: Mean score using nested cross-validation is: 0.574 ± 0.116
SVC: Mean score using nested cross-validation is: 0.618 ± 0.111
XGBoost: Mean score using nested cross-validation is: 0.645 ± 0.102


# LF for fmri+75 regions. Logistic regression

In [53]:
fmri_and_75reg = pd.read_csv('fmri_cobre_75regions.csv')
fmri = pd.read_csv('fmri_without_duplicates.csv')
reg75 = pd.read_csv('75regions_without_duplicates.csv')

In [54]:
reg75

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,17 1ary Visual Anterior (BA17a),20 Posterior Fusiform (BA37m),27 2ary/3ary Visual Anterior (BA18/19a),29 Precuneus (BA7m),41 Angular (BA39),10 2ary/3ary Visual Posterior (BA18/19p),14 1ary Visual Posterior (BA17p),19 Posterior Middle Temporal (BA37-pMT),target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,103.948415,117.754574,97.951265,90.466617,74.563782,85.600885,116.860705,90.293591,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,135.452954,113.762617,99.168800,84.776504,84.182907,121.552500,116.242818,98.573632,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,171.634672,128.278143,165.282009,98.787972,107.424595,139.489700,146.718045,122.982998,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,148.392664,146.273071,145.256096,113.367215,130.451834,136.680432,139.514955,114.017828,1,A00000909
4,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,107.198542,79.971999,108.201993,103.729257,106.887096,118.010823,124.285205,94.770384,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,121.907310,140.090268,150.628304,120.923228,107.951374,133.114588,144.640263,93.140918,1,A00037649
119,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,130.509874,127.331203,119.756711,110.778300,87.421152,143.013958,117.791072,103.933863,0,A00037665
120,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,151.471291,124.232667,124.079747,75.842177,101.550097,124.739328,149.489989,104.270125,1,A00037854
121,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,114.575210,99.474787,135.220648,94.860538,94.225460,98.043687,121.035822,97.334191,1,A00038441


In [55]:
fmri

,"('Precentral_R', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_R')","('Frontal_Sup_R', 'Precentral_L')","('Frontal_Sup_R', 'Precentral_R')","('Frontal_Sup_R', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Precentral_L')","('Frontal_Sup_Orb_L', 'Precentral_R')","('Frontal_Sup_Orb_L', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Frontal_Sup_R')",...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,0.590307,0.215914,0.304414,0.116425,0.165778,0.255798,0.031173,-0.113507,0.038880,0.170122,...,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,1,A00000368
1,0.600053,0.450454,0.407950,0.554324,0.457576,0.747958,0.101570,-0.024283,0.218750,0.244049,...,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,1,A00000456
2,0.640427,0.425068,0.470462,0.313035,0.265838,0.615625,-0.134986,-0.242692,-0.040470,0.093006,...,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,1,A00000838
3,0.427308,0.239172,0.101873,0.266489,0.325015,0.580501,0.027387,-0.153969,0.092569,-0.007205,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1,A00000909
4,0.664985,0.299166,0.073119,0.407197,0.240695,0.669216,0.168402,0.111856,0.097451,0.082961,...,-0.142375,0.048878,0.311628,0.185092,0.333022,0.338988,0.568694,0.423569,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0.787778,0.418824,0.420073,0.536880,0.531874,0.776564,0.222002,0.235727,-0.130599,0.042551,...,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,1,A00037649
119,0.837095,0.699350,0.567253,0.628090,0.484317,0.816644,0.515332,0.440825,0.597178,0.574797,...,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,0,A00037665
120,0.626127,0.367332,0.178448,0.252641,0.384334,0.654329,0.114585,0.223127,-0.013945,0.164300,...,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,1,A00037854
121,0.457744,0.486875,0.204815,0.321213,0.284081,0.572457,-0.062898,-0.092247,-0.064799,-0.135443,...,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,1,A00038441


In [57]:
# Path to the CSV files

file1_path = '75regions_without_duplicates.csv'
file2_path = 'fmri_without_duplicates.csv'

# Read the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Concatenate the dataframes based on the ID column
concatenated_df = pd.merge(df1, df2, on='ID')

# Save the concatenated dataframe to a new CSV file
output_path = 'fmri_and_75regions_no_duplicates.csv'
concatenated_df.to_csv(output_path, index=False)

print("CSV files concatenated based on the ID column and saved as 'concatenated.csv'.")

CSV files concatenated based on the ID column and saved as 'concatenated.csv'.


In [58]:
file3 = pd.read_csv('fmri_and_75regions_no_duplicates.csv')
file3

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,"('Vermis_10', 'Cerebelum_10_L')","('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target_y
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,0.175623,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,1
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,0.192315,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,1
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,0.069086,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,1
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,0.170549,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1
4,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,0.048278,-0.142375,0.048878,0.311628,0.185092,0.333022,0.338988,0.568694,0.423569,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,0.526304,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,1
119,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,-0.333526,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,0
120,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,0.483774,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,1
121,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,0.618097,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,1


In [67]:
fmri_and_75reg = pd.read_csv('fmri_and_75regions_no_duplicates.csv')
fmri = pd.read_csv('fmri_without_duplicates.csv')
reg75 = pd.read_csv('75regions_without_duplicates.csv')

In [75]:
fmri_and_75reg

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1,A00000909
4,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,-0.142375,0.048878,0.311628,0.185092,0.333022,0.338988,0.568694,0.423569,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,1,A00037649
119,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,0,A00037665
120,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,1,A00037854
121,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,1,A00038441


In [69]:
# custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

In [70]:
   # fit clf1 with df1
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,77) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(77,6748) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


eclf = VotingClassifier(estimators=[('df1-clf1', pipe1), ('df2-clf2', pipe2)], voting='soft')

In [ ]:
param_grid = [
    {
        'df1-clf1__clf__penalty': ['l1', 'l2'],
        'df1-clf1__clf__C': [0.1, 1, 10],
        'df1-clf1__clf__fit_intercept': [True, False],
        'df1-clf1__clf__solver': ['liblinear'],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__penalty': ['l1', 'l2'],
        'df2-clf2__clf__C': [0.1, 1, 10],
        'df2-clf2__clf__fit_intercept': [True, False],
        'df2-clf2__clf__solver': ['liblinear'],
        'df2-clf2__clf__random_state': [1380]
    }
]

In [73]:
from sklearn.model_selection import cross_val_score, KFold

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.706 ± 0.076


# LF for fmri+75 regions. Random forest

In [86]:
# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define the pipelines
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,77) )),
    ( "scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', RandomForestClassifier(random_state=1380))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(77,6748) )),
    ( "scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', RandomForestClassifier(random_state=1380))
])



# Create a VotingClassifier with all three pipelines
eclf = VotingClassifier(estimators=[
    ('df1-clf1', pipe1),
    ('df2-clf2', pipe2)
], voting='soft')

# Load the data and target
target =fmri_and_75reg.target
data_late =fmri_and_75reg.drop(columns=['target', 'ID'])

# Define the parameter grid for each pipeline
param_grid = [
    {
        'df1-clf1__clf__n_estimators': [10, 50, 100],
        'df1-clf1__clf__max_depth': [3, 5, 7, 9],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__n_estimators': [10, 50, 100],
        'df2-clf2__clf__max_depth': [3, 5, 7, 9],
        'df2-clf2__clf__random_state': [1380]
    }

]


# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
print(f"The mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.685 ± 0.084


# LF for fmri+75 regions. SVC

In [87]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import VotingClassifier
import numpy as np

# Define your custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define three pipelines, each with an SVC classifier
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0,77))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(77,6748))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])


# Create a VotingClassifier with all three pipelines
eclf = VotingClassifier(estimators=[
    ('df1-clf1', pipe1),
    ('df2-clf2', pipe2)
], voting='soft')

# Load your data and target
target =fmri_and_75reg.target
data_late =fmri_and_75reg.drop(columns=['target', 'ID'])

# Define the parameter grid for each pipeline
param_grid = [
    {
        'df1-clf1__clf__C': [0.01, 0.1, 1],
        'df1-clf1__clf__class_weight': ['balanced', None],
        'df1-clf1__clf__kernel': ['linear', 'rbf'],
        'df1-clf1__clf__probability': [True],
        'df1-clf1__clf__random_state': [seed]
    },
    {
        'df2-clf2__clf__C': [0.01, 0.1, 1],
        'df2-clf2__clf__class_weight': ['balanced', None],
        'df2-clf2__clf__kernel': ['linear', 'rbf'],
        'df2-clf2__clf__probability': [True],
        'df2-clf2__clf__random_state': [seed]
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
print(f"The mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.674 ± 0.128


# LF for fmri+75 regions. XGBoost

In [88]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
import numpy as np

# Load the data
target =fmri_and_75reg.target
data_late =fmri_and_75reg.drop(columns=['target', 'ID'])

# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self


# Define the pipeline
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0,77))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(77,6748))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])


# Define the parameter grid
param_grid = [
    {
        'clf__learning_rate': [0.1],
        'clf__max_depth': [6, 7, 3, 10],
        'clf__scale_pos_weight': [1],
        'clf__subsample': [1, 0.5],
        'clf__alpha': [0],
        'clf__random_state': [1380],
        'clf__use_label_encoder': [False],
        'clf__objective': ['binary:logistic']
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=pipe1, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=1, scoring='f1_macro')
print(f"The mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")


The mean score using nested cross-validation is: 0.659 ± 0.086


# Morphometry scores

In [96]:
morph = pd.read_csv('morphometry_no_duplicates.csv')

morph

,Left-Lateral-Ventricle_NVoxels,Left-Inf-Lat-Vent_NVoxels,Left-Cerebellum-White-Matter_NVoxels,Left-Cerebellum-Cortex_NVoxels,Left-Thalamus-Proper_NVoxels,Left-Caudate_NVoxels,Left-Putamen_NVoxels,Left-Pallidum_NVoxels,3rd-Ventricle_NVoxels,4th-Ventricle_NVoxels,...,supramarginal_CurvInd_lh,frontalpole_CurvInd_lh,temporalpole_CurvInd_lh,transversetemporal_CurvInd_lh,insula_CurvInd_lh,file,ID,target,Sex,Unnamed: 931
0,18693,274,14134,50431,7221,3499,4535,1996,792,1462,...,6.8,0.8,1.3,0.4,3.7,morphometry_A00007409,A00007409,1,male,NaN
1,2489,181,12436,50238,7236,3330,4463,2107,546,1044,...,6.2,1.0,1.2,1.2,7.9,morphometry_A00022500,A00022500,0,female,NaN
2,7704,231,13415,54324,8037,2523,4709,2499,1032,1662,...,6.9,1.7,1.2,0.8,4.9,morphometry_A00024953,A00024953,0,male,NaN
3,10600,304,13585,62141,7035,3302,5281,2180,854,1831,...,5.1,0.8,0.7,1.4,3.5,morphometry_A00001251,A00001251,0,male,NaN
4,10407,209,24048,59909,8349,2858,4452,2191,1066,1759,...,7.5,0.8,0.8,1.0,4.5,morphometry_A00037495,A00037495,1,male,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,5177,187,17224,58244,7490,3822,4449,2083,857,2315,...,5.6,1.2,0.9,0.5,5.0,morphometry_A00022727,A00022727,1,male,NaN
119,4339,148,12149,47024,7194,3371,4094,1854,496,1055,...,7.2,1.1,1.1,0.6,4.2,morphometry_A00022653,A00022653,1,female,NaN
120,10667,193,14434,55105,7806,4303,5323,2368,636,1077,...,7.7,0.8,1.0,0.9,5.9,morphometry_A00010150,A00010150,1,male,NaN
121,5873,242,17772,52836,7745,3865,5033,2401,703,1115,...,5.7,0.8,0.8,0.7,5.2,morphometry_A00016723,A00016723,0,female,NaN


# EF for morphometry

In [97]:
target =reg75_without_duplicates.target
data =reg75_without_duplicates.drop(columns=['target', 'ID'])


# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Logistic Regression: Mean score using nested cross-validation is: 0.619 ± 0.083
Random Forest: Mean score using nested cross-validation is: 0.574 ± 0.116
SVC: Mean score using nested cross-validation is: 0.618 ± 0.111
XGBoost: Mean score using nested cross-validation is: 0.645 ± 0.102


# Concat fmri+morp

In [98]:
# Path to the CSV files

file1_path = 'morphometry_no_duplicates.csv'
file2_path = 'fmri_without_duplicates.csv'

# Read the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Concatenate the dataframes based on the ID column
concatenated_df = pd.merge(df1, df2, on='ID')

# Save the concatenated dataframe to a new CSV file
output_path = 'fmri_and_morp_123.csv'
concatenated_df.to_csv(output_path, index=False)

print("CSV files concatenated based on the ID column and saved as 'concatenated.csv'.")

CSV files concatenated based on the ID column and saved as 'concatenated.csv'.


In [4]:
fmri_and_morph = pd.read_csv('fmri_and_morp_123.csv')
fmri = pd.read_csv('fmri_without_duplicates.csv')
morph = pd.read_csv('morphometry_no_duplicates.csv')

In [112]:
morph

,Left-Lateral-Ventricle_NVoxels,Left-Inf-Lat-Vent_NVoxels,Left-Cerebellum-White-Matter_NVoxels,Left-Cerebellum-Cortex_NVoxels,Left-Thalamus-Proper_NVoxels,Left-Caudate_NVoxels,Left-Putamen_NVoxels,Left-Pallidum_NVoxels,3rd-Ventricle_NVoxels,4th-Ventricle_NVoxels,...,superiorfrontal_CurvInd_lh,superiorparietal_CurvInd_lh,superiortemporal_CurvInd_lh,supramarginal_CurvInd_lh,frontalpole_CurvInd_lh,temporalpole_CurvInd_lh,transversetemporal_CurvInd_lh,insula_CurvInd_lh,ID,target
0,18693,274,14134,50431,7221,3499,4535,1996,792,1462,...,11.3,7.7,4.6,6.8,0.8,1.3,0.4,3.7,A00007409,1
1,2489,181,12436,50238,7236,3330,4463,2107,546,1044,...,13.1,6.7,6.0,6.2,1.0,1.2,1.2,7.9,A00022500,0
2,7704,231,13415,54324,8037,2523,4709,2499,1032,1662,...,15.4,6.7,7.1,6.9,1.7,1.2,0.8,4.9,A00024953,0
3,10600,304,13585,62141,7035,3302,5281,2180,854,1831,...,11.9,7.6,4.5,5.1,0.8,0.7,1.4,3.5,A00001251,0
4,10407,209,24048,59909,8349,2858,4452,2191,1066,1759,...,10.5,7.0,6.1,7.5,0.8,0.8,1.0,4.5,A00037495,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,5177,187,17224,58244,7490,3822,4449,2083,857,2315,...,9.2,6.5,4.2,5.6,1.2,0.9,0.5,5.0,A00022727,1
119,4339,148,12149,47024,7194,3371,4094,1854,496,1055,...,10.5,6.7,4.9,7.2,1.1,1.1,0.6,4.2,A00022653,1
120,10667,193,14434,55105,7806,4303,5323,2368,636,1077,...,18.9,11.3,9.3,7.7,0.8,1.0,0.9,5.9,A00010150,1
121,5873,242,17772,52836,7745,3865,5033,2401,703,1115,...,10.5,7.7,4.2,5.7,0.8,0.8,0.7,5.2,A00016723,0


In [113]:
fmri

,"('Precentral_R', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_R')","('Frontal_Sup_R', 'Precentral_L')","('Frontal_Sup_R', 'Precentral_R')","('Frontal_Sup_R', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Precentral_L')","('Frontal_Sup_Orb_L', 'Precentral_R')","('Frontal_Sup_Orb_L', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Frontal_Sup_R')",...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,0.590307,0.215914,0.304414,0.116425,0.165778,0.255798,0.031173,-0.113507,0.038880,0.170122,...,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,1,A00000368
1,0.600053,0.450454,0.407950,0.554324,0.457576,0.747958,0.101570,-0.024283,0.218750,0.244049,...,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,1,A00000456
2,0.640427,0.425068,0.470462,0.313035,0.265838,0.615625,-0.134986,-0.242692,-0.040470,0.093006,...,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,1,A00000838
3,0.427308,0.239172,0.101873,0.266489,0.325015,0.580501,0.027387,-0.153969,0.092569,-0.007205,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1,A00000909
4,0.664985,0.299166,0.073119,0.407197,0.240695,0.669216,0.168402,0.111856,0.097451,0.082961,...,-0.142375,0.048878,0.311628,0.185092,0.333022,0.338988,0.568694,0.423569,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0.787778,0.418824,0.420073,0.536880,0.531874,0.776564,0.222002,0.235727,-0.130599,0.042551,...,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,1,A00037649
119,0.837095,0.699350,0.567253,0.628090,0.484317,0.816644,0.515332,0.440825,0.597178,0.574797,...,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,0,A00037665
120,0.626127,0.367332,0.178448,0.252641,0.384334,0.654329,0.114585,0.223127,-0.013945,0.164300,...,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,1,A00037854
121,0.457744,0.486875,0.204815,0.321213,0.284081,0.572457,-0.062898,-0.092247,-0.064799,-0.135443,...,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,1,A00038441


In [114]:
fmri_and_morph

,Left-Lateral-Ventricle_NVoxels,Left-Inf-Lat-Vent_NVoxels,Left-Cerebellum-White-Matter_NVoxels,Left-Cerebellum-Cortex_NVoxels,Left-Thalamus-Proper_NVoxels,Left-Caudate_NVoxels,Left-Putamen_NVoxels,Left-Pallidum_NVoxels,3rd-Ventricle_NVoxels,4th-Ventricle_NVoxels,...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,18693,274,14134,50431,7221,3499,4535,1996,792,1462,...,0.009590,0.172087,0.269736,0.165370,0.231563,0.136910,0.292152,0.399644,0,A00007409
1,2489,181,12436,50238,7236,3330,4463,2107,546,1044,...,0.191281,0.215860,0.195709,0.140152,0.267767,0.253835,0.444850,0.429113,1,A00022500
2,7704,231,13415,54324,8037,2523,4709,2499,1032,1662,...,-0.075162,-0.159331,-0.014370,-0.116377,-0.036073,0.292674,0.350473,0.483932,1,A00024953
3,10600,304,13585,62141,7035,3302,5281,2180,854,1831,...,0.103264,0.416603,0.083171,-0.177938,-0.020037,0.221657,0.559467,0.414580,1,A00001251
4,10407,209,24048,59909,8349,2858,4452,2191,1066,1759,...,-0.358174,0.548469,0.329123,0.301622,0.417169,0.293198,0.532769,0.347501,0,A00037495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,5177,187,17224,58244,7490,3822,4449,2083,857,2315,...,-0.022749,0.241897,0.182758,0.083769,0.155714,0.181779,0.366814,0.282371,0,A00022727
119,4339,148,12149,47024,7194,3371,4094,1854,496,1055,...,-0.246840,0.274223,0.202504,-0.238776,0.290823,0.209776,0.312112,0.302832,0,A00022653
120,10667,193,14434,55105,7806,4303,5323,2368,636,1077,...,-0.309410,0.015058,0.503868,0.520389,0.573108,0.567371,0.587601,0.686093,0,A00010150
121,5873,242,17772,52836,7745,3865,5033,2401,703,1115,...,-0.421263,0.216793,0.231175,-0.010506,0.010442,0.207897,0.454866,0.598393,1,A00016723


# LF morph+fmri. Logistic Regression

In [115]:
# custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self
    
       # fit clf1 with df1
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,929) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(929,7599) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


eclf = VotingClassifier(estimators=[('df1-clf1', pipe1), ('df2-clf2', pipe2)], voting='soft')

target =fmri_and_morph.target
data_late =fmri_and_morph.drop(columns=['target', 'ID'])

param_grid = [
    {
        'df1-clf1__clf__penalty': ['l1', 'l2'],
        'df1-clf1__clf__C': [0.1, 1, 10],
        'df1-clf1__clf__fit_intercept': [True, False],
        'df1-clf1__clf__solver': ['liblinear'],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__penalty': ['l1', 'l2'],
        'df2-clf2__clf__C': [0.1, 1, 10],
        'df2-clf2__clf__fit_intercept': [True, False],
        'df2-clf2__clf__solver': ['liblinear'],
        'df2-clf2__clf__random_state': [1380]
    }
]

from sklearn.model_selection import cross_val_score, KFold

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.683 ± 0.098


# LF morph+fmri. Random Forest

In [6]:
# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define the pipelines
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,929) )),
    ( "scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', RandomForestClassifier(random_state=1380))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(929,7599) )),
    ( "scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', RandomForestClassifier(random_state=1380))
])



# Create a VotingClassifier with all three pipelines
eclf = VotingClassifier(estimators=[
    ('df1-clf1', pipe1),
    ('df2-clf2', pipe2)
], voting='soft')

# Load the data and target
target =fmri_and_morph.target
data_late =fmri_and_morph.drop(columns=['target', 'ID'])

# Define the parameter grid for each pipeline
param_grid = [
    {
        'df1-clf1__clf__n_estimators': [10, 50, 100],
        'df1-clf1__clf__max_depth': [3, 5, 7, 9],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__n_estimators': [10, 50, 100],
        'df2-clf2__clf__max_depth': [3, 5, 7, 9],
        'df2-clf2__clf__random_state': [1380]
    }

]


# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
print(f"The mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.618 ± 0.183


# LF morph+fmri. SVC

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import VotingClassifier
import numpy as np

# Define your custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define three pipelines, each with an SVC classifier
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0,929))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(929,7599))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])


# Create a VotingClassifier with all three pipelines
eclf = VotingClassifier(estimators=[
    ('df1-clf1', pipe1),
    ('df2-clf2', pipe2)
], voting='soft')

# Load your data and target
target =fmri_and_morph.target
data_late =fmri_and_morph.drop(columns=['target', 'ID'])

# Define the parameter grid for each pipeline
param_grid = [
    {
        'df1-clf1__clf__C': [0.01, 0.1, 1],
        'df1-clf1__clf__class_weight': ['balanced', None],
        'df1-clf1__clf__kernel': ['linear', 'rbf'],
        'df1-clf1__clf__probability': [True],
        'df1-clf1__clf__random_state': [seed]
    },
    {
        'df2-clf2__clf__C': [0.01, 0.1, 1],
        'df2-clf2__clf__class_weight': ['balanced', None],
        'df2-clf2__clf__kernel': ['linear', 'rbf'],
        'df2-clf2__clf__probability': [True],
        'df2-clf2__clf__random_state': [seed]
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
print(f"The mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.606 ± 0.153


# LF morph+fmri. XGBoost

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
import numpy as np

# Load the data
target =fmri_and_morph.target
data_late =fmri_and_morph.drop(columns=['target', 'ID'])

# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self


# Define the pipeline
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0,929))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(929,7599))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])


# Define the parameter grid
param_grid = [
    {
        'clf__learning_rate': [0.1],
        'clf__max_depth': [6, 7, 3, 10],
        'clf__scale_pos_weight': [1],
        'clf__subsample': [1, 0.5],
        'clf__alpha': [0],
        'clf__random_state': [1380],
        'clf__use_label_encoder': [False],
        'clf__objective': ['binary:logistic']
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=pipe1, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=1, scoring='f1_macro')
print(f"The mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.531 ± 0.104


# EF for fmri+morph

In [10]:
target =fmri_and_morph.target
data =fmri_and_morph.drop(columns=['target', 'ID'])


# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


Logistic Regression: Mean score using nested cross-validation is: 0.700 ± 0.126


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


Random Forest: Mean score using nested cross-validation is: 0.647 ± 0.146


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


SVC: Mean score using nested cross-validation is: 0.583 ± 0.186
XGBoost: Mean score using nested cross-validation is: 0.617 ± 0.121


# fmri+morph+reg75

In [12]:
# Path to the CSV files

file1_path = '75reg_without_duplicates.csv'
file2_path = 'fmri_and_morp_123.csv'

# Read the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Concatenate the dataframes based on the ID column
concatenated_df = pd.merge(df1, df2, on='ID')

# Save the concatenated dataframe to a new CSV file
output_path = 'fmri_and_morp_and_75reg_123.csv'
concatenated_df.to_csv(output_path, index=False)

print("CSV files concatenated based on the ID column and saved as 'concatenated.csv'.")

CSV files concatenated based on the ID column and saved as 'concatenated.csv'.


In [4]:
fmri_and_morph_and_75reg = pd.read_csv('fmri_and_morp_and_75reg_123.csv')
morph = pd.read_csv('morphometry_no_duplicates.csv')
fmri = pd.read_csv('fmri_without_duplicates.csv')
reg75 = pd.read_csv('75reg_without_duplicates.csv')

In [5]:
fmri_and_morph_and_75reg

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1,A00000909
4,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,-0.142375,0.048878,0.311628,0.185092,0.333022,0.338988,0.568694,0.423569,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,1,A00037649
119,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,0,A00037665
120,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,1,A00037854
121,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,1,A00038441


In [6]:
reg75

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,17 1ary Visual Anterior (BA17a),20 Posterior Fusiform (BA37m),27 2ary/3ary Visual Anterior (BA18/19a),29 Precuneus (BA7m),41 Angular (BA39),10 2ary/3ary Visual Posterior (BA18/19p),14 1ary Visual Posterior (BA17p),19 Posterior Middle Temporal (BA37-pMT),target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,103.948415,117.754574,97.951265,90.466617,74.563782,85.600885,116.860705,90.293591,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,135.452954,113.762617,99.168800,84.776504,84.182907,121.552500,116.242818,98.573632,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,171.634672,128.278143,165.282009,98.787972,107.424595,139.489700,146.718045,122.982998,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,148.392664,146.273071,145.256096,113.367215,130.451834,136.680432,139.514955,114.017828,1,A00000909
4,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,107.198542,79.971999,108.201993,103.729257,106.887096,118.010823,124.285205,94.770384,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,121.907310,140.090268,150.628304,120.923228,107.951374,133.114588,144.640263,93.140918,1,A00037649
119,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,130.509874,127.331203,119.756711,110.778300,87.421152,143.013958,117.791072,103.933863,0,A00037665
120,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,151.471291,124.232667,124.079747,75.842177,101.550097,124.739328,149.489989,104.270125,1,A00037854
121,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,114.575210,99.474787,135.220648,94.860538,94.225460,98.043687,121.035822,97.334191,1,A00038441


In [7]:
morph

,Left-Lateral-Ventricle_NVoxels,Left-Inf-Lat-Vent_NVoxels,Left-Cerebellum-White-Matter_NVoxels,Left-Cerebellum-Cortex_NVoxels,Left-Thalamus-Proper_NVoxels,Left-Caudate_NVoxels,Left-Putamen_NVoxels,Left-Pallidum_NVoxels,3rd-Ventricle_NVoxels,4th-Ventricle_NVoxels,...,superiorfrontal_CurvInd_lh,superiorparietal_CurvInd_lh,superiortemporal_CurvInd_lh,supramarginal_CurvInd_lh,frontalpole_CurvInd_lh,temporalpole_CurvInd_lh,transversetemporal_CurvInd_lh,insula_CurvInd_lh,ID,target
0,18693,274,14134,50431,7221,3499,4535,1996,792,1462,...,11.3,7.7,4.6,6.8,0.8,1.3,0.4,3.7,A00007409,1
1,2489,181,12436,50238,7236,3330,4463,2107,546,1044,...,13.1,6.7,6.0,6.2,1.0,1.2,1.2,7.9,A00022500,0
2,7704,231,13415,54324,8037,2523,4709,2499,1032,1662,...,15.4,6.7,7.1,6.9,1.7,1.2,0.8,4.9,A00024953,0
3,10600,304,13585,62141,7035,3302,5281,2180,854,1831,...,11.9,7.6,4.5,5.1,0.8,0.7,1.4,3.5,A00001251,0
4,10407,209,24048,59909,8349,2858,4452,2191,1066,1759,...,10.5,7.0,6.1,7.5,0.8,0.8,1.0,4.5,A00037495,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,5177,187,17224,58244,7490,3822,4449,2083,857,2315,...,9.2,6.5,4.2,5.6,1.2,0.9,0.5,5.0,A00022727,1
119,4339,148,12149,47024,7194,3371,4094,1854,496,1055,...,10.5,6.7,4.9,7.2,1.1,1.1,0.6,4.2,A00022653,1
120,10667,193,14434,55105,7806,4303,5323,2368,636,1077,...,18.9,11.3,9.3,7.7,0.8,1.0,0.9,5.9,A00010150,1
121,5873,242,17772,52836,7745,3865,5033,2401,703,1115,...,10.5,7.7,4.2,5.7,0.8,0.8,0.7,5.2,A00016723,0


In [8]:
fmri

,"('Precentral_R', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_L')","('Frontal_Sup_L', 'Precentral_R')","('Frontal_Sup_R', 'Precentral_L')","('Frontal_Sup_R', 'Precentral_R')","('Frontal_Sup_R', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Precentral_L')","('Frontal_Sup_Orb_L', 'Precentral_R')","('Frontal_Sup_Orb_L', 'Frontal_Sup_L')","('Frontal_Sup_Orb_L', 'Frontal_Sup_R')",...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,0.590307,0.215914,0.304414,0.116425,0.165778,0.255798,0.031173,-0.113507,0.038880,0.170122,...,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,1,A00000368
1,0.600053,0.450454,0.407950,0.554324,0.457576,0.747958,0.101570,-0.024283,0.218750,0.244049,...,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,1,A00000456
2,0.640427,0.425068,0.470462,0.313035,0.265838,0.615625,-0.134986,-0.242692,-0.040470,0.093006,...,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,1,A00000838
3,0.427308,0.239172,0.101873,0.266489,0.325015,0.580501,0.027387,-0.153969,0.092569,-0.007205,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1,A00000909
4,0.664985,0.299166,0.073119,0.407197,0.240695,0.669216,0.168402,0.111856,0.097451,0.082961,...,-0.142375,0.048878,0.311628,0.185092,0.333022,0.338988,0.568694,0.423569,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,0.787778,0.418824,0.420073,0.536880,0.531874,0.776564,0.222002,0.235727,-0.130599,0.042551,...,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,1,A00037649
119,0.837095,0.699350,0.567253,0.628090,0.484317,0.816644,0.515332,0.440825,0.597178,0.574797,...,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,0,A00037665
120,0.626127,0.367332,0.178448,0.252641,0.384334,0.654329,0.114585,0.223127,-0.013945,0.164300,...,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,1,A00037854
121,0.457744,0.486875,0.204815,0.321213,0.284081,0.572457,-0.062898,-0.092247,-0.064799,-0.135443,...,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,1,A00038441


# EF 75 reg+morph+fmri

In [10]:
target =fmri_and_morph_and_75reg.target
data =fmri_and_morph_and_75reg.drop(columns=['target', 'ID'])


# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


Logistic Regression: Mean score using nested cross-validation is: 0.708 ± 0.082


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


Random Forest: Mean score using nested cross-validation is: 0.718 ± 0.122


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


SVC: Mean score using nested cross-validation is: 0.692 ± 0.124
XGBoost: Mean score using nested cross-validation is: 0.728 ± 0.076


# LF 75 reg+morph+fmri. Logistic regression

In [23]:
# custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self
    
       # fit clf1 with df1
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,77) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(77,1006) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])

pipe3 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(1006,7674) )),
    ( "scaler", StandardScaler()),
    ('clf', LogisticRegression(random_state=seed))
    ])


eclf = VotingClassifier(estimators=[('df1-clf1', pipe1), ('df2-clf2', pipe2), ('df3-clf3', pipe3)], voting='soft')

target =fmri_and_morph_and_75reg.target
data_late =fmri_and_morph_and_75reg.drop(columns=['target', 'ID'])

param_grid = [
    {
        'df1-clf1__clf__penalty': ['l1', 'l2'],
        'df1-clf1__clf__C': [0.1, 1, 10],
        'df1-clf1__clf__fit_intercept': [True, False],
        'df1-clf1__clf__solver': ['liblinear'],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__penalty': ['l1', 'l2'],
        'df2-clf2__clf__C': [0.1, 1, 10],
        'df2-clf2__clf__fit_intercept': [True, False],
        'df2-clf2__clf__solver': ['liblinear'],
        'df2-clf2__clf__random_state': [1380]
    },
        {
        'df3-clf3__clf__penalty': ['l1', 'l2'],
        'df3-clf3__clf__C': [0.1, 1, 10],
        'df3-clf3__clf__fit_intercept': [True, False],
        'df3-clf3__clf__solver': ['liblinear'],
        'df3-clf3__clf__random_state': [1380]
    }
    
]

from sklearn.model_selection import cross_val_score, KFold

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(
    estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1
)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1,scoring='f1_macro')
print(f"The mean score using nested cross-validation is: "
      f"{test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.690 ± 0.094


# LF 75 reg+morph+fmri. Random forest

In [10]:
# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define the pipelines
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(0,77) )),
    ( "scaler", StandardScaler()),
    ('clf', RandomForestClassifier(random_state=1380))
    ])


pipe2 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(77,1006) )),
    ( "scaler", StandardScaler()),
    ('clf', RandomForestClassifier(random_state=1380))
    ])

pipe3 = Pipeline([
    ('col_extract', ColumnExtractor( cols=range(1006,7674) )),
    ( "scaler", StandardScaler()),
    ('clf', RandomForestClassifier(random_state=1380))
    ])


# Create a VotingClassifier with all three pipelines
eclf = VotingClassifier(estimators=[
    ('df1-clf1', pipe1),
    ('df2-clf2', pipe2),
    ('df3-clf3', pipe3)
    
], voting='soft')

# Load the data and target
target =fmri_and_morph_and_75reg.target
data_late =fmri_and_morph_and_75reg.drop(columns=['target', 'ID'])

# Define the parameter grid for each pipeline
param_grid = [
    {
        'df1-clf1__clf__n_estimators': [10, 50, 100],
        'df1-clf1__clf__max_depth': [3, 5, 7, 9],
        'df1-clf1__clf__random_state': [1380]
    },
    {
        'df2-clf2__clf__n_estimators': [10, 50, 100],
        'df2-clf2__clf__max_depth': [3, 5, 7, 9],
        'df2-clf2__clf__random_state': [1380]
    },
    {
        'df3-clf3__clf__n_estimators': [10, 50, 100],
        'df3-clf3__clf__max_depth': [3, 5, 7, 9],
        'df3-clf3__clf__random_state': [1380]
    }
    

]


# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
print(f"The mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.638 ± 0.105


# LF 75 reg+morph+fmri. SVC

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.ensemble import VotingClassifier
import numpy as np

# Define your custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self

# Define three pipelines, each with an SVC classifier
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0,77))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(77,1006))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])

pipe3 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(1006,7674))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', SVC(random_state=seed, probability=True))
])

# Create a VotingClassifier with all three pipelines
eclf = VotingClassifier(estimators=[
    ('df1-clf1', pipe1),
    ('df2-clf2', pipe2),
    ('df3-clf3', pipe3)
], voting='soft')

# Load your data and target
target =fmri_and_morph_and_75reg.target
data_late =fmri_and_morph_and_75reg.drop(columns=['target', 'ID'])

# Define the parameter grid for each pipeline
param_grid = [
    {
        'df1-clf1__clf__C': [0.01, 0.1, 1],
        'df1-clf1__clf__class_weight': ['balanced', None],
        'df1-clf1__clf__kernel': ['linear', 'rbf'],
        'df1-clf1__clf__probability': [True],
        'df1-clf1__clf__random_state': [seed]
    },
    {
        'df2-clf2__clf__C': [0.01, 0.1, 1],
        'df2-clf2__clf__class_weight': ['balanced', None],
        'df2-clf2__clf__kernel': ['linear', 'rbf'],
        'df2-clf2__clf__probability': [True],
        'df2-clf2__clf__random_state': [seed]
    },
        {
        'df3-clf3__clf__C': [0.01, 0.1, 1],
        'df3-clf3__clf__class_weight': ['balanced', None],
        'df3-clf3__clf__kernel': ['linear', 'rbf'],
        'df3-clf3__clf__probability': [True],
        'df3-clf3__clf__random_state': [seed]
    }
    
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=eclf, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')
print(f"The mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.706 ± 0.074


# LF 75 reg+morph+fmri. XGBoost

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from xgboost import XGBClassifier
import numpy as np

# Load the data
target =fmri_and_morph_and_75reg.target
data_late =fmri_and_morph_and_75reg.drop(columns=['target', 'ID'])

# Custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        col_list = []
        for c in self.cols:
            col_list.append(X.iloc[:, c:c+1])
        return np.concatenate(col_list, axis=1)

    def fit(self, X, y=None):
        return self


# Define the pipeline
pipe1 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(0,77))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])

pipe2 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(77,1006))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])

pipe3 = Pipeline([
    ('col_extract', ColumnExtractor(cols=range(1006,7674))),
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=74)),
    ('clf', XGBClassifier(random_state=seed, use_label_encoder=False, objective='binary:logistic'))
])


# Define the parameter grid
param_grid = [
    {
        'clf__learning_rate': [0.1],
        'clf__max_depth': [6, 7, 3, 10],
        'clf__scale_pos_weight': [1],
        'clf__subsample': [1, 0.5],
        'clf__alpha': [0],
        'clf__random_state': [1380],
        'clf__use_label_encoder': [False],
        'clf__objective': ['binary:logistic']
    }
]

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=1380)

# Inner cross-validation for parameter search
model = GridSearchCV(estimator=pipe1, param_grid=param_grid, cv=inner_cv, n_jobs=-1)

# Outer cross-validation to compute the testing score
test_score = cross_val_score(model, data_late, target, cv=outer_cv, n_jobs=1, scoring='f1_macro')
print(f"The mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

The mean score using nested cross-validation is: 0.651 ± 0.101


# EF 75reg+fmri

In [18]:
# Path to the CSV files

file1_path = '75reg_without_duplicates.csv'
file2_path = 'fmri_without_duplicates.csv'

# Read the CSV files
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)

# Concatenate the dataframes based on the ID column
concatenated_df = pd.merge(df1, df2, on='ID')

# Save the concatenated dataframe to a new CSV file
output_path = 'fmri_and_75reg_123.csv'
concatenated_df.to_csv(output_path, index=False)

print("CSV files concatenated based on the ID column and saved as 'concatenated.csv'.")

CSV files concatenated based on the ID column and saved as 'concatenated.csv'.


In [21]:
fmri_with_75_reg = pd.read_csv('fmri_and_75_reg.csv')
fmri_with_75_reg.drop_duplicates()
fmri_with_75_reg

,39 Frontopolar (BA10fp),38 Anterior Prefrontal (BA10),46 Prefrontal Medial (BA10m),7 Orbitofrontal (BA47),34 Dorsolateral Prefrontal (BA9),35 Dorsolateral Prefrontal Medial (BA9m),40 Cingulate Anterior (BA32g),44 Dorsolateral Prefrontal (BA46),"45 Ventrolateral Prefrontal, Broca (BA45)",50 Orbitofrontal (BA11),...,"('Vermis_10', 'Cerebelum_10_R')","('Vermis_10', 'Vermis_1_2')","('Vermis_10', 'Vermis_3')","('Vermis_10', 'Vermis_4_5')","('Vermis_10', 'Vermis_6')","('Vermis_10', 'Vermis_7')","('Vermis_10', 'Vermis_8')","('Vermis_10', 'Vermis_9')",target,ID
0,58.866298,32.037922,65.388297,78.246759,56.629863,66.967725,66.493130,82.406754,74.978737,112.924148,...,0.258102,0.078604,-0.023349,-0.069403,-0.019248,0.273587,0.323913,0.156899,1,A00000368
1,79.917920,60.905715,50.692958,125.995181,95.138743,96.950000,96.471839,108.433811,65.292060,107.932340,...,-0.015206,0.073420,0.150168,-0.086901,0.118768,0.432538,0.563580,0.360348,1,A00000456
2,115.118735,90.996473,56.787130,161.254288,101.813830,84.993841,74.291767,105.749643,129.896290,162.709481,...,0.012403,-0.201170,0.272472,-0.022364,-0.157833,-0.103954,0.078484,0.040241,1,A00000838
3,48.265023,55.398722,58.455889,120.482916,59.180747,106.721808,93.969429,52.593249,67.652083,110.079885,...,-0.081796,0.289600,0.185260,0.197285,0.071489,0.085987,0.124079,0.504530,1,A00000909
4,44.988870,35.195800,72.324544,106.788054,26.731926,74.340262,81.345844,69.795069,69.216825,117.076847,...,-0.142375,0.048878,0.311628,0.185092,0.333022,0.338988,0.568694,0.423569,1,A00001181
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,63.175807,88.194589,84.751006,128.039491,79.521503,69.837557,86.539192,108.341193,99.972632,146.323215,...,0.287224,0.044076,-0.042164,-0.138347,0.116347,-0.128351,-0.010187,0.564968,1,A00037649
119,94.953496,67.716278,85.526353,109.462153,94.592965,94.221063,101.793120,99.317570,103.662488,130.630941,...,0.013771,-0.297782,0.018670,-0.199247,0.221397,-0.145123,0.218707,-0.338076,0,A00037665
120,106.324507,91.073551,66.061469,117.908016,58.678237,74.761711,95.303333,104.618375,101.099159,131.033936,...,0.286602,0.362241,0.263336,0.268425,0.312150,0.147089,0.466238,0.586675,1,A00037854
121,44.191215,30.267923,29.013437,125.549416,26.616800,47.758473,88.583859,82.862007,96.347417,128.475395,...,0.507622,0.348055,0.089759,-0.062223,0.229889,0.489779,0.626306,0.491610,1,A00038441


In [22]:
target =fmri_with_75_reg.target
data =fmri_with_75_reg.drop(columns=['target', 'ID'])


# Define the pipeline for feature selection and model training

pipeline_lr = Pipeline([
    ("scaler", StandardScaler()),
   # ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", LogisticRegression(random_state=1380))
])

pipeline_rf = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", RandomForestClassifier(random_state=1380))
])

pipeline_svc = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", SVC(random_state=1380))
])

pipeline_xgb = Pipeline([
    ("scaler", StandardScaler()),
    ("fs", SelectFromModel(estimator=LogisticRegression(random_state=seed), max_features=75)),
    ("model", XGBClassifier(random_state=1380))
])

# Define the parameter grids for each model
param_grid_lr= {
    'model__penalty': ['l1', 'l2'],
    'model__C': [0.1, 1, 10],
    'model__fit_intercept': [True, False],
    'model__solver': ['liblinear'],
    'model__random_state': [1380]
}

param_grid_rf = {
    'model__n_estimators': [10, 50, 100],
    'model__max_depth': [3, 5, 7, 9],
    'model__random_state': [1380]
}

param_grid_svc = {
    'model__C': [0.01, 0.1, 1],
    'model__class_weight': ['balanced', None],
    'model__kernel': ['linear', 'rbf'],
    'model__probability': [True],
    'model__random_state': [1380]
}

param_grid_xgb = {
    'model__learning_rate': [0.1],
    'model__max_depth': [6, 7, 3, 10],
    'model__scale_pos_weight': [1],
    'model__subsample': [1, 0.5],
    'model__alpha': [0],
    'model__random_state': [1380],
    'model__use_label_encoder': [False],
    'model__objective': ['binary:logistic'],
    
}

# Declare the inner and outer cross-validation strategies
inner_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
outer_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# Models to evaluate
models = {
    'Logistic Regression': (pipeline_lr, param_grid_lr),
    'Random Forest': (pipeline_rf, param_grid_rf),
    'SVC': (pipeline_svc, param_grid_svc),
    'XGBoost': (pipeline_xgb, param_grid_xgb)
}

# Perform nested cross-validation for each model
for model_name, (pipeline, param_grid) in models.items():
    # Inner cross-validation for parameter search
    model = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=inner_cv, n_jobs=-1, scoring='f1_macro')

    # Outer cross-validation to compute the testing score
    test_score = cross_val_score(model, data, target, cv=outer_cv, n_jobs=-1, scoring='f1_macro')

    print(f"{model_name}: Mean score using nested cross-validation is: {test_score.mean():.3f} ± {test_score.std():.3f}")

Logistic Regression: Mean score using nested cross-validation is: 0.709 ± 0.101
Random Forest: Mean score using nested cross-validation is: 0.722 ± 0.113
SVC: Mean score using nested cross-validation is: 0.723 ± 0.109


/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)
/opt/conda/lib/python3.9/site-packages/sklearn/utils/deprecation.py:101: FutureWarning: Attribute _pairwise was deprecated in version 0.24 and will be removed in 1.1 (renaming of 0.26).
  warnings.warn(msg, category=FutureWarning)


XGBoost: Mean score using nested cross-validation is: 0.736 ± 0.081
